In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd

In [5]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/MLP/mini_project/review_project/NewsResult_20210101-20220301.xlsx")

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
df.head(3)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100601e+06,20220301,한국경제,민지혜,"올리브영,빅데이터 기반 AI 스타트업 '로켓뷰' 인수",경제>산업_기업,IT_과학>콘텐츠,경제>유통,NaN,NaN,NaN,김화경,NaN,"올리브영,마켓인사이트,삼성전자,디지털사업본부","올리브영,빅데이터,로켓뷰,AI,스타트업,인수,기사,13:34,마켓인사이트,게재,기사...","올리브영,로켓뷰,ai,스타트업,큐레이션,빅데이터,마켓인사이트,화장품,상품명,온라인몰...",이 기사는 03월 01일 13:34 “마켓인사이트”에 게재된 기사입니다. \n \n...,https://www.hankyung.com/finance/article/20220...,NaN
1,2.100801e+06,20220301,아시아경제,김유리,"CJ올리브영, AI 스타트업 '로켓뷰' 인수 상품 추천 엔진 내재화",경제>산업_기업,경제>유통,IT_과학>인터넷_SNS,NaN,NaN,NaN,김화경,La,"CJ올리브영,올리브영,애플,C랩,AI,삼성전자,디지털사업본부","CJ올리브영,로켓뷰,AI,스타트업,인수,상품,내재화,추천,엔진,디지털,투자,AI,인...","올리브영,ai,로켓뷰,빅데이터,화장품,스타트업,큐레이션,삼성전자,온라인몰,내재화",[아시아경제 김유리 기자] CJ올리브영이 올해 첫 디지털 투자 분야로 인공지능(AI...,https://www.asiae.co.kr/article/20220301213547...,NaN
2,1.100401e+06,20220301,동아일보,김민범,"‘AI 올리브영’ 등장 CJ올리브영, 상품 추천 기술 특화 스타트업 ‘로켓뷰’ 인수",경제>산업_기업,경제>유통,IT_과학>인터넷_SNS,NaN,NaN,NaN,김화경,"La,창사","CJ올리브영,올리브영,로켓뷰,C랩,AI,삼성전자","AI,올리브영,등장,CJ올리브영,상품,추천,특화,스타트업,로켓뷰,인수,초개인화,큐레...","올리브영,ai,로켓뷰,빅데이터,삼성전자,스타트업,큐레이션,내재화,화장품,1억",CJ올리브영이 올해 첫 디지털 투자 분야로 인공지능(AI)을 선택했다. AI가 소비...,https://www.donga.com/news/article/all/2022030...,NaN


In [8]:
keywords = pd.DataFrame(df)
keywords['본문']

0       이 기사는 03월 01일 13:34 “마켓인사이트”에 게재된 기사입니다. \n \n...
1       [아시아경제 김유리 기자] CJ올리브영이 올해 첫 디지털 투자 분야로 인공지능(AI...
2       CJ올리브영이 올해 첫 디지털 투자 분야로 인공지능(AI)을 선택했다. AI가 소비...
3       [헤럴드경제=이정아 기자] 올리브영은 빅데이터 기반 인공지능(AI) 스타트업 ‘로켓...
4       올리브영 로고[사진=CJ올리브영] \n \n \n \nCJ올리브영이 올해 첫 디지털...
                              ...                        
1452    [머니투데이 오정은 기자] [서경배 회장 "고객 중심으로 변해야 산다" vs 차석용...
1453    올해 창립 50주년을 맞이한 현대백화점그룹이 “2030년 매출 40조 시대를 열겠다...
1454    지난해에는 신종 코로나바이러스 감염증(코로나19) 타격을 대기업도 피해가지 못했다....
1455    헤어케어 전문 브랜드 닥터포헤어(Dr FORHAIR)가 브랜드 모델 현빈과 ‘헤어에...
1456    인천 쿠팡 메가물류센터에서 직원들이 물품 분류작업을 하고 있다. 쿠팡 제공 코로나1...
Name: 본문, Length: 1457, dtype: object

In [9]:
docs = df['본문'].tolist()
docs

["이 기사는 03월 01일 13:34 “마켓인사이트”에 게재된 기사입니다. \n \n헬스&뷰티(H&B) 스토어 올리브영은 빅데이터 기반 인공지능(AI) 스타트업 '로켓뷰'를 인수했다고 1일 밝혔다. 이번 인수를 통해 올리브영은 초개인화 큐레이션 서비스를 본격적으로 선보일 계획이다. \n \n올해 '혁신 성장'을 전략 키워드로 내건 올리브영은 주요 전략인 디지털 ..",
 "[아시아경제 김유리 기자] CJ올리브영이 올해 첫 디지털 투자 분야로 인공지능(AI)을 택했다. \n \n올리브영은 빅데이터 기반 AI 스타트업 '로켓뷰'를 인수했다고 1일 밝혔다. 연간 1억건을 웃도는 고객 구매 데이터를 축적하고 있는 올리브영은 이번 로켓뷰 인수를 통해 빅데이터와 AI 기반의 상품 추천 엔진을 장착, 이를 통한 초개인화 큐레이션 서비스를..",
 'CJ올리브영이 올해 첫 디지털 투자 분야로 인공지능(AI)을 선택했다. AI가 소비자에게 최적화된 상품을 제안하는 ‘AI 올리브영’이 등장할 전망이다. \n \nCJ올리브영은 빅데이터 기반 AI 스타트업 ‘로켓뷰’를 인수했다고 1일 밝혔다. 연간 1억 건을 웃도는 고객 구매 데이터를 축적하고 있는 올리브영은 이번 로켓뷰 인수를 통해 빅데이터와 AI 기반 상..',
 "[헤럴드경제=이정아 기자] 올리브영은 빅데이터 기반 인공지능(AI) 스타트업 ‘로켓뷰’를 인수했다고 1일 밝혔다. \n \n 로켓뷰는 2017년부터 스마트폰 카메라로 화장품 상품명을 촬영하면 최저가와 상품 속성, 성분 등의 상품 정보를 알려주는 '찍검(찍고 검색)' 서비스 애플리케이션을 선보이고 있다. \n \n 딥러닝 기반 광학 문자 인식(OCR) 수집 솔루..",
 '올리브영 로고[사진=CJ올리브영] \n \n \n \nCJ올리브영이 올해 첫 디지털 투자 분야로 AI(인공지능)를 택했다. \n \nCJ올리브영은 빅데이터 기반 AI 스타트업 ‘로켓뷰’를 인수했다고 1일 밝혔다. \n \n연간 1억건을 웃도는 고객 구매 데이터를 축적하고 있는 올리브영은 이번 로켓

In [10]:
n_topics = 30

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(docs)
svd = TruncatedSVD(n_components=n_topics, n_iter=100)

U = svd.fit_transform(tfidf)
Vt = svd.components_

In [11]:
vocab =  tfidf_vect.get_feature_names_out()
print(len(vocab))

17610


In [12]:
n = 5
for i, topic in enumerate(Vt):
    print("Topic {} :".format(i), [ (vocab[i], topic[i].round(3)) for i in topic.argsort()[:-n-1:-1]])

Topic 0 : [('밝혔다', 0.216), ('올리브영', 0.21), ('cj올리브영은', 0.188), ('올리브영은', 0.149), ('브랜드', 0.145)]
Topic 1 : [('피부', 0.228), ('민감', 0.205), ('진정', 0.203), ('브랜드', 0.192), ('화장품', 0.178)]
Topic 2 : [('어워즈', 0.257), ('올리브영', 0.218), ('페스타', 0.21), ('2021', 0.16), ('ddp', 0.155)]
Topic 3 : [('어워즈', 0.23), ('2021', 0.169), ('페스타', 0.161), ('올리브영', 0.124), ('진정', 0.122)]
Topic 4 : [('오늘드림', 0.207), ('배송', 0.147), ('픽업', 0.137), ('서비스', 0.136), ('상품을', 0.133)]
Topic 5 : [('알뜰폰', 0.544), ('lg유플러스는', 0.157), ('파트너스', 0.138), ('중소', 0.131), ('lg유플러스가', 0.13)]
Topic 6 : [('it', 0.226), ('ai', 0.14), ('올리브영은', 0.132), ('인력을', 0.123), ('나선다', 0.117)]
Topic 7 : [('it', 0.302), ('인력을', 0.165), ('공개', 0.147), ('창사', 0.143), ('정보기술', 0.141)]
Topic 8 : [('ai', 0.241), ('건강', 0.19), ('기반', 0.165), ('로켓뷰', 0.158), ('추천', 0.156)]
Topic 9 : [('선물', 0.265), ('멤버십', 0.144), ('설날', 0.129), ('아이템', 0.121), ('제안하는', 0.12)]
Topic 10 : [('멤버십', 0.271), ('1000만', 0.149), ('리뷰', 0.14), ('회원', 0.124), ('1000만명을', 0.12